In [286]:
import numpy as np
import os
import shlex, subprocess
import sys,threading,subprocess
import time
from sedpy import observate
import extinction

In [14]:
def Gaussian(x,mu,sigma,amp):
    return amp * np.exp(-0.5*(x-mu)**2/sigma**2) / (np.sqrt(2*np.pi)*sigma) #<- do we need to normalize this?

In [305]:
def calc_modelMag(galaxy_parameters, redshift, filternames):
    # simulate the spectra in the rest frame
    wavelength,flux = run_PEGASE(galaxy_parameters)
    
    # apply redshift and get the magnitude in specified filters
    magnitudes = flux_to_mag(wavelength,flux,redshift,filternames)

    return magnitudes

In [306]:
def run_PEGASE(galaxy_parameters):
    #read galaxy_parameters
    age, mass, expo_timescale, late_mass = galaxy_parameters
    age_to_use = int(round(age))
    
    #start write subprocess. This code may need to be moved into the top level function, I'm not sure how it works with multiproccessing
    proc=subprocess.Popen('/bin/sh',stdout=subprocess.PIPE,stdin=subprocess.PIPE,stderr=subprocess.STDOUT)
    pout=proc.stdout
    pin=proc.stdin

    '''def outLoop():
        running=True
        while(running):
            line=pout.readline().decode(sys.stdout.encoding)
            print(line,end='')
            running='\n' in line
        print('Finished')

    threading.Thread(target=outLoop).start()'''

    def write(inp):
        pin.write(bytes(inp+'\n','utf-8'))
        pin.flush()
    
    #writing scenarios.txt and ages.txt

    file_name = ''

    for p in galaxy_parameters:
        file_name = file_name + str(p) + '__'
    
    identifier = str(np.random.randint(10000))
    file_name = file_name + identifier

    SSPs_name = 'Chabrier_SSPs.txt'
    spectra_name = file_name + 'spectra.txt'
    scenario_name = file_name + 'scenarios.txt'
    color_name = file_name + 'colors.txt'
    spectra_ages_name = file_name + 'spectra_ages.txt'
    
    scenario_text = 'SSPs_set = "' + SSPs_name + '''"

    ages_file = '''+ spectra_ages_name +'''

    SF_type = "exponential"
    SF_expo_timescale = '''+ str(expo_timescale) +'''
    SF_expo_mass = '''+ str(mass) +'''

    reserv_init_mass = 1.
    infall_expo_timescale = 1000
    SF_ISM_timescale = 3000
    nebular_emission = .true.
    extinction = .true.
    inclin_averaged = .true.
    dust_emission = .true.

    SF_type(2) = "exponential"
    SF_expo_mass(2) = '''+ str(late_mass) +'''
    SF_begin_time(2) = '''+ str(age_to_use - 100) +'''
    SF_expo_timescale(2) = 100
    
    spectra_file = "'''+ spectra_name +'''"
    return

    '''

    f = open('../scenarios_dir/' + scenario_name,'w')
    f.write(scenario_text)
    f.close()

    #write SSPs_ages file
    f = open('../ages_dir/' + spectra_ages_name,'w')
    f.write(str(age_to_use))
    f.close()
    
    #create spectra file
    write('./spectra')
    write(scenario_name)
    proc.communicate()
    
    '''wait_file = file_name + 'waiting.txt'
    time.sleep(5)
    write('touch ' + wait_file)
    path = './'
    while os.path.isfile('./'+wait_file) == False:
        time.sleep(1)
    write('rm ' + wait_file)'''
    
    #create color file
    '''write('./colors')
    write(spectra_name)
    write(color_name)

    wait_file = file_name + 'waiting.txt'
    time.sleep(1)
    write('touch ' + wait_file)
    path = './'
    while os.path.isfile('./'+wait_file) == False:
        time.sleep(1)
    write('rm ' + wait_file)'''
    
    #read spectra file
    A = np.ravel(np.loadtxt('../spectra_dir/'+spectra_name,skiprows=146,max_rows=476,unpack=False))
    B = np.loadtxt('../spectra_dir/'+spectra_name,skiprows=622,max_rows=1,unpack=False)
    lambda_cont = np.append(A,B)

    lambda_line = np.ravel(np.loadtxt('../spectra_dir/'+spectra_name,skiprows=624,max_rows=401,usecols=(1),unpack=False))

    A = np.ravel(np.loadtxt('../spectra_dir/'+spectra_name,skiprows=1067,max_rows=238,unpack=False))
    B = np.ravel(np.loadtxt('../spectra_dir/'+spectra_name,skiprows=1305,max_rows=1,unpack=False))
    L_cont = np.append(A,B)

    A = np.ravel(np.loadtxt('../spectra_dir/'+spectra_name,skiprows=1307,max_rows=40,unpack=False))
    B = np.ravel(np.loadtxt('../spectra_dir/'+spectra_name,skiprows=1347,max_rows=1,unpack=False))
    L_line = np.append(A,B)

    f = open('../spectra_dir/'+spectra_name,'r')
    lines = f.readlines()
    stellar_mass = float(lines[1034].split()[0])
    f.close()
    
    #combine emission lines and continuum
    v = 10**2.2 #velocity dispersion in km/s
    c = 299792 #c in km/s
    L_full = L_cont
    for i in range(len(L_line)):
        sigma = lambda_line[i]*((np.sqrt(1+v/c)/np.sqrt(1-v/c))-1)
        spec_line = Gaussian(lambda_cont,lambda_line[i],sigma,L_line[i])
        L_full = L_full + spec_line
    
    #convert to flux
    distance = 30856775812799586000 #10 parsec in cm
    f_full = (mass / stellar_mass) * L_full / (4*np.pi*distance**2)
    
    #delete all the created files
    os.remove('../spectra_dir/'+spectra_name)
    os.remove('../ages_dir/'+spectra_ages_name)
    os.remove('../scenarios_dir/'+scenario_name)
    
    
    
    return lambda_cont, f_full

In [307]:
def flux_to_mag(wavelength,flux,redshift,filternames):
    #redshift wavelengths
    r_wavelength = wavelength*(1+redshift)
    
    #dust extinction
    #flux = extinction.apply(extinction.ccm89(wavelength, A_V, 3.1), flux)
    
    #get return functions for wavelengths
    return_U = np.loadtxt('../calib_dir/UX_B90.txt',skiprows=5,unpack=True)
    return_B = np.loadtxt('../calib_dir/B_B90.txt',skiprows=6,unpack=True)
    return_V = np.loadtxt('../calib_dir/V_B90.txt',skiprows=5,unpack=True)
    return_R = np.loadtxt('../calib_dir/RC.txt',skiprows=5,unpack=True)
    return_I = np.loadtxt('../calib_dir/IC.txt',skiprows=5,unpack=True)

    return_u = np.loadtxt('../calib_dir/u_SDSS.txt',skiprows=5,unpack=True)
    return_g = np.loadtxt('../calib_dir/g_SDSS.txt',skiprows=5,unpack=True)
    return_r = np.loadtxt('../calib_dir/r_SDSS.txt',skiprows=5,unpack=True)
    return_i = np.loadtxt('../calib_dir/i_SDSS.txt',skiprows=5,unpack=True)
    return_z = np.loadtxt('../calib_dir/z_SDSS.txt',skiprows=5,unpack=True)

    return_J = np.loadtxt('../calib_dir/J_2MASS.txt',skiprows=5,unpack=True)
    return_H = np.loadtxt('../calib_dir/H_2MASS.txt',skiprows=5,unpack=True)
    return_K = np.loadtxt('../calib_dir/Ks_2MASS.txt',skiprows=5,unpack=True)

    #filters = ['U','B','V','R','I','u','g','r','i','z','J','H','K']
    return_func = {
        'U' : return_U,
        'B' : return_B,
        'V' : return_V,
        'R' : return_R,
        'I' : return_I,
        'u' : return_u,
        'g' : return_g,
        'r' : return_r,
        'i' : return_i,
        'z' : return_z,
        'J' : return_J,
        'H' : return_H,
        'K' : return_K
    }
    calib_type = {
        'U' : 'Vega',
        'B' : 'Vega',
        'V' : 'Vega',
        'R' : 'Vega',
        'I' : 'Vega',
        'u' : 'AB',
        'g' : 'AB',
        'r' : 'AB',
        'i' : 'AB',
        'z' : 'AB',
        'J' : 'Vega',
        'H' : 'Vega',
        'K' : 'Vega'
    }

    filt_mag = {}
    for f in filternames:

        filt = observate.Filter(f, data=(return_func[f][0,:], return_func[f][1,:]))
        if calib_type[f] == 'AB':
            filt_mag[f] = filt.ab_mag(r_wavelength, flux)
            
        elif calib_type[f] == 'Vega':
            filt_mag[f] = filt.vega_mag(r_wavelength, flux)
    
    return filt_mag


In [303]:
galaxy_parameters = [10006, 1, 400, 0.01]
redshift = 0
#A_V = 1
filternames = ['u','g','r','i','z']
start_time = time.time()
filt_mag = calc_modelMag(galaxy_parameters, redshift, filternames)
end_time = time.time()
print(end_time-start_time)

7.426332712173462


In [304]:
filt_mag['z']

5.437774558442023